# INITIALIZATION

In [ ]:
import numpy as np
import tensorflow as tf

# Import MNIST data
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("./tmp/data/", one_hot=True)

In [ ]:
# Parameters
learning_rate = 0.1
num_steps = 500
batch_size = 128
display_step = 100

# Network Parameters
n_hidden_1 = 256 # 1st layer number of neurons
n_hidden_2 = 256 # 2nd layer number of neurons
num_input = 784 # MNIST data input (img shape: 28*28)
num_classes = 10 # MNIST total classes (0-9 digits)

# tf Graph input
X = tf.placeholder("float", [None, num_input])
Y = tf.placeholder("float", [None, num_classes])

In [ ]:
# Store layers weight & bias
weights = {
    'h1': tf.Variable(tf.random_normal([num_input, n_hidden_1])),
    'h2': tf.Variable(tf.random_normal([n_hidden_1, n_hidden_2])),
    'out': tf.Variable(tf.random_normal([n_hidden_2, num_classes]))
}
biases = {
    'b1': tf.Variable(tf.random_normal([n_hidden_1])),
    'b2': tf.Variable(tf.random_normal([n_hidden_2])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

In [ ]:
# Create model
def neural_net(x):
    # Hidden fully connected layer with 256 neurons
    layer_1 = tf.add(tf.matmul(x, weights['h1']), biases['b1'])
    # Hidden fully connected layer with 256 neurons
    layer_2 = tf.add(tf.matmul(layer_1, weights['h2']), biases['b2'])
    # Output fully connected layer with a neuron for each class
    out_layer = tf.matmul(layer_2, weights['out']) + biases['out']
    return out_layer

In [ ]:
# Construct model
logits = neural_net(X)

# Define loss and optimizer
loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

In [ ]:
# Evaluate model (with test logits, for dropout to be disabled)
correct_pred = tf.equal(tf.argmax(logits, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

In [ ]:
# Initialize the variables (i.e. assign their default value)
init = tf.global_variables_initializer()

# Save and Restore
saver = tf.train.Saver()

# SAVE VARIABLES

In [ ]:
# Set value loading operations
def param_value_load_ops(param_tensor_list):
    """

    Args:
        param_tensor_list: list of trainable parameters

    Returns:
        load_ops_list: list of ops
        load_ph_list: list of placeholders
    """
    load_ops_list = []
    load_ph_list = []

    # For each parameter
    for param_tensor in param_tensor_list:
        # Make a copy of placeholder and load it
        param_load_ph = tf.placeholder(dtype=param_tensor.dtype,
                                       shape=param_tensor.shape)
        load_ph_list.append(param_load_ph)
        # Load ops with placeholder assigned to it
        load_ops_list.append(param_tensor.assign(param_load_ph))

    return load_ops_list, load_ph_list

In [ ]:
# Start training
with tf.Session() as sess:
    # Run the initializer
    sess.run(init)
    
    # save initial weights and biases
    # w_init and b_init will be in dict
    w_init = sess.run(weights)
    b_init = sess.run(biases)
    init_param_value_list = []
    # collect only values from w_init and b_init
    # list of keys will keep the order
    for key in ['h1', 'h2', 'out']:
        init_param_value_list.append(w_init[key])
    for key in ['b1', 'b2', 'out']:
        init_param_value_list.append(b_init[key])
    
    # Train!
    for step in range(1, num_steps+1):
        batch_x, batch_y = mnist.train.next_batch(batch_size)
        # Run optimization
        sess.run(train_op, feed_dict={X: batch_x, Y: batch_y})
        if step % display_step == 0 or step == 1:
            # Calculate batch loss and accuracy
            loss, acc = sess.run([loss_op, accuracy], feed_dict={X: batch_x,
                                                                 Y: batch_y})
            print("Step " + str(step) + ", Minibatch Loss= " + \
                  "{:.4f}".format(loss) + ", Training Accuracy= " + \
                  "{:.3f}".format(acc))
            save_path = saver.save(sess, "./tmp/model.ckpt")
            print("Model saved in file: %s" % save_path)

    print("Optimization Finished!")

    # Save trained weights and biases
    w_trnd = sess.run(weights)
    b_trnd = sess.run(biases)
    trnd_param_value_list = []
    for key in ['h1', 'h2', 'out']:
        trnd_param_value_list.append(w_trnd[key])
    for key in ['b1', 'b2', 'out']:
        trnd_param_value_list.append(b_trnd[key])

    # Calculate accuracy for MNIST test images
    print("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={X: mnist.test.images,
                                      Y: mnist.test.labels}))

Step 1, Minibatch Loss= 9307.6553, Training Accuracy= 0.477
Model saved in file: ./tmp/model.ckpt
Step 100, Minibatch Loss= 433.5188, Training Accuracy= 0.859
Model saved in file: ./tmp/model.ckpt
Step 200, Minibatch Loss= 97.8582, Training Accuracy= 0.875
Model saved in file: ./tmp/model.ckpt
Step 300, Minibatch Loss= 82.5491, Training Accuracy= 0.906
Model saved in file: ./tmp/model.ckpt
Step 400, Minibatch Loss= 41.6152, Training Accuracy= 0.914
Model saved in file: ./tmp/model.ckpt
Step 500, Minibatch Loss= 32.8969, Training Accuracy= 0.914
Model saved in file: ./tmp/model.ckpt
Optimization Finished!
('Testing Accuracy:', 0.8495)


# RESTORE VARIABLES

In [ ]:
# Get list of trainable parameters
# 이렇게 하면 default graph 상에 있는 Varialble 들의 pointer 를 꺼내는 셈
param_tensor_list = tf.trainable_variables()

In [ ]:
# Build load operation
# 사용하기 쉽게 list 로 정리해주는 것이긴 한데,
# 이렇게 해도 결국은 pointer 로 사용하겠다는 것임
load_ops_list, load_ph_list = param_value_load_ops(param_tensor_list)

# DO LINEAR COMBINATION AND EVALUATE

In [ ]:
with tf.Session() as sess:
    sess.run(init)
    
    # linearly combine init and trained parameters
    alphas = np.arange(0.0,2.01,.05)
    errors = []
    for alpha in alphas:
        feed_dict = {}
        for load_ph, init_value, trnd_value in zip(load_ph_list, init_param_value_list, trnd_param_value_list):
            # linearly combine values
            # 이해: 이 부분이 문법이 이해가 안 됨
            feed_dict[load_ph] = init_value*(1.-alpha) + trnd_value*alpha

        # 이해: 이 부분이 문법이 이해가 안 됨
        sess.run(load_ops_list,
                 feed_dict=feed_dict)

        # do eval by using the graph
        # Calculate accuracy for MNIST test images
        acc = sess.run(accuracy, feed_dict={X: mnist.test.images,
                                            Y: mnist.test.labels})
        print("Testing Accuracy:", acc )
        errors.append((1-acc)*100)

# PLOT

In [ ]:
import matplotlib
import matplotlib.pyplot as plt

%matplotlib inline

In [ ]:
plt.plot(alphas, errors)

plt.xlabel('alpha')
plt.ylabel('error(%)')
plt.title('The Long and Winding Road for MNIST')
plt.grid(True)
# plt.savefig("mnist_journey.png")
plt.show()

# CHECK